In [ ]:
import torch
import torch.nn as nn
import numpy as np

class ConvNet1D(nn.Module):
    def __init__(self, input_size, input_channels):
        super(ConvNet1D, self).__init__()

        self.n_filters_1 = 64
        self.n_filters_2 = 128
        self.n_filters_3 = 256
        self.n_filters_4 = 512

        self.kernel_size_1 = 3
        self.kernel_size_2 = 3
        self.kernel_size_3 = 3
        self.kernel_size_4 = 3

        self.hidden_size = 512

        self.conv1, output_dim = self._construct_layer(input_size=input_size, 
                                                       in_channels=input_channels,
                                                       out_channels=self.n_filters_1,
                                                       kernel_size=self.kernel_size_1,
                                                       stride=1)
        # print("Output 1:", output_dim)
        self.conv2, output_dim = self._construct_layer(input_size=output_dim, 
                                                       in_channels=self.n_filters_1,
                                                       out_channels=self.n_filters_2,
                                                       kernel_size=self.kernel_size_2,
                                                       stride=1,
                                                       pool_width=2)
        # print("Output 2:", output_dim)

        self.conv3, output_dim = self._construct_layer(input_size=output_dim, 
                                                       in_channels=self.n_filters_2,
                                                       out_channels=self.n_filters_3,
                                                       kernel_size=self.kernel_size_4,
                                                       stride=1)
        # print("Output 3:", output_dim)
        self.conv4, output_dim = self._construct_layer(input_size=output_dim, 
                                                       in_channels=self.n_filters_3,
                                                       out_channels=self.n_filters_4,
                                                       kernel_size=self.kernel_size_4,
                                                       stride=1,
                                                       pool_width=4)
        # print("Output 4:", output_dim)
        self.final_dim = output_dim  * self.n_filters_4
        self.fc = nn.Sequential(
            nn.Linear(self.final_dim, self.hidden_size),
            nn.Linear(self.hidden_size, 1)
        )

    def _construct_layer(self, input_size, in_channels, out_channels, kernel_size, stride, pool_width=-1):

        padding = (kernel_size - 1) // 2
        output_dim = (input_size + 2 * padding - kernel_size) / stride + 1
        if output_dim % 1 != 0:
            raise ValueError('Invalid kernel/stride parameter values')

        sublayers = [
            nn.Conv1d(in_channels=in_channels,
                      out_channels=out_channels,
                      kernel_size=kernel_size,
                      stride=stride,
                      padding=(kernel_size - 1) // 2),
            nn.BatchNorm1d(out_channels),
            nn.ReLU()
        ]

        if pool_width > 0:
            sublayers.append(nn.MaxPool1d(kernel_size=pool_width))
            pool_output_dim = (output_dim - pool_width) / pool_width + 1
            if pool_output_dim % 1 != 0:
                raise ValueError('Invalid pool width for output dim {}'.format(pool_output_dim))
            else:
                output_dim = pool_output_dim

        return nn.Sequential(*sublayers), int(output_dim)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = x.view(-1, self.final_dim)
        x = self.fc(x)
        x = torch.sigmoid(x)
        return x
